# トピックへの自動ラベリング


## コンテンツを取得

In [4]:
import requests
from bs4 import BeautifulSoup

# url = "https://ledge.ai/authorinterview-book-bert-int/"
url = "https://www.yomiuri.co.jp/world/20230104-OYT1T50056/"
# url = "https://www3.nhk.or.jp/kansai-news/20230104/2000069636.html"
# url = "https://news.yahoo.co.jp/articles/fad0c4f41d46b686e0566bf10e4c016a641a9dab"
# url = "https://news.yahoo.co.jp/articles/4d2d14fd1ca1dc9b134c5f493896a7e30fc1e781"
res = requests.get(url)

soup = BeautifulSoup(res.content, "lxml")
for tg in ["script", "noscript", "meta"]:
    try:
        soup.find(tg).replace_with(" ")
    except:
        pass
soup.get_text()[:200]

'\n\n \n\n\n \n\n砲撃受けたウクライナ東部の露軍臨時兵舎、死者８９人に…露側「原因」は兵士が携帯使用と主張 : 読売新聞オンライン\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'

### 簡易クレンジング

In [5]:
import re

def clean_text(text: str):
    contents = []
    for txt in re.split(r"(。|\n)", text):
        txt = txt.strip().replace("\u200b", "").replace("\u3000", " ")
        txt = re.sub(r"\n+", "\n", txt)
        txt = re.sub(r"([\W])\1+", " ", txt)
        if not txt:
            continue
        # contents.append(txt)
        # contents.extend(txt.split("\n"))
        contents.append(txt.split("\n")[-1])
    return contents

text = soup.get_text()
contents = clean_text(text)
contents[:5]

['砲撃受けたウクライナ東部の露軍臨時兵舎、死者８９人に…露側「原因」は兵士が携帯使用と主張 : 読売新聞オンライン',
 'すべて',
 'ログイン・登録',
 'ログイン',
 '新規登録']

In [6]:
# # pickup wiki data to analyze topic
# wdb = WikiDb(mode="train")
# records = wdb.select_by_document_id(document_ids=[
#     "Doc01GN7P2K02Y2FC8BXHMZ5V1A34",    # 南部煎餅
#     "Doc01GN7P3C1ZPVGB8AXJC75M9ZQ9",    # 龍が如く OF THE END
#     "Doc01GN7P3C0YPAWH4KT16P7NNQBK",    # 自動運転車
#     "Doc01GN7P3BS2JNYS3HYV0RM1TT7T",    # デュケイン大学
#     "Doc01GN7P3BK88YCXKS64H6G2HK0Y",    # 深蒸し茶
#     ])
# X: TextSequences = [WikiRecord(*rec).paragraph.splitlines() for rec in records]
# pipe_topic.fit(X)

## トピックモデルのパイプラインを構築

In [7]:
import joblib

from app.component.models.model import TextSequences
from app.component.models.pipeline import Pipeline
from app.component.models.vectorizer import VectorizerBoW, VectorizerWord2vec
from app.domain.models.tokenizer import TokenizerWord
from app.domain.models.topic_model import TopicModel
from app.infra.wikidb import WikiDb, WikiRecord

In [8]:
pipe_topic = Pipeline(
    steps=[
        (TokenizerWord(use_stoppoes=True), None),
        (VectorizerBoW(), None),
        (TopicModel(n_topic=10, n_epoch=2000), None),
    ],
    name="pipe_topic_sample",
    do_print=True,
)


In [9]:
# X: TextSequences = [[snt] for snt in contents]
X: TextSequences = [contents]
# pipe_topic[:1].fit(X).transform(X)        # for debugging
pipe_topic.fit(X)

2023/01/08 20:03:23 INFO Start to fit n_step=0 model=JpTokenizerMeCab(use_stoppoes=True, filterpos=[], use_orgform=False)
2023/01/08 20:03:23 INFO End to fit n_step=0 model=JpTokenizerMeCab(use_stoppoes=True, filterpos=[], use_orgform=False)
2023/01/08 20:03:23 INFO Start to transform n_step=0 model=JpTokenizerMeCab(use_stoppoes=True, filterpos=[], use_orgform=False)
2023/01/08 20:03:32 INFO End to transform n_step=0 model=JpTokenizerMeCab(use_stoppoes=True, filterpos=[], use_orgform=False)
2023/01/08 20:03:32 INFO Start to fit n_step=1 model=VectorizerBoW()
2023/01/08 20:03:32 INFO End to fit n_step=1 model=VectorizerBoW()
2023/01/08 20:03:32 INFO Start to transform n_step=1 model=VectorizerBoW()
2023/01/08 20:03:32 INFO End to transform n_step=1 model=VectorizerBoW()
2023/01/08 20:03:32 INFO Start to fit n_step=2 model=TopicModel(n_topic=10, n_epoch=2000)
2023/01/08 20:03:32 INFO End to fit n_step=2 model=TopicModel(n_topic=10, n_epoch=2000)
2023/01/08 20:03:32 INFO Start to transfor

Pipeline(steps=[(JpTokenizerMeCab(use_stoppoes=True, filterpos=[], use_orgform=False), None), (VectorizerBoW(), None), (TopicModel(n_topic=10, n_epoch=2000), None)], name=pipe_topic_sample, do_print=True, args=(), kwargs={})

In [10]:
model_bow: VectorizerBoW = pipe_topic.get_model(1)
model_topic: TopicModel = pipe_topic.get_model(-1)
model_bow, model_topic


(VectorizerBoW(), TopicModel(n_topic=10, n_epoch=2000))

## Word2Vec モデルをロード

In [11]:
# # use original pretrained newsvec model (with only noun and verb tokens)
pipe_newsvec = joblib.load("data/pipe_newsvec.gz")
pipe_newsvec
model_vectorizer: VectorizerWord2vec = pipe_newsvec.get_model(-1)
model_vectorizer
w2v = model_vectorizer.model

In [8]:
# # use original pretrained wikivec model (with only noun and verb tokens)
# pipe_wikivec = joblib.load("data/pipe_wikivec.noun-verb.gz")
# pipe_wikivec
# model_vectorizer: VectorizerWord2vec = pipe_wikivec.get_model(-1)
# model_vectorizer
# w2v = model_vectorizer.model

In [9]:
# # use shiroyagi's word2vec pretrained model
# from gensim.models.word2vec import Word2Vec
# w2v = Word2Vec.load("data/word2vec.gensim.model")
# w2v

In [12]:
len(w2v.wv.key_to_index)

455

In [13]:
# 各トピックの確率の合計が 1 になることを確認しておく
model_topic.get_topic_probabilities().sum(-1)

array([1.        , 1.0000001 , 1.        , 1.        , 1.0000001 ,
       1.        , 0.99999994, 1.        , 1.0000001 , 1.0000001 ],
      dtype=float32)

In [12]:
def pickup_topic_words(model_topic: TopicModel, topn: int = -1):
    # topn: 各トピックの上位の単語の数
    topics = []
    for topic_probs in model_topic.get_topic_probabilities():
        indices = topic_probs.argsort()[::-1][:topn]
        topic = [(model_bow.vocab[idx], topic_probs[idx]) for idx in indices]
        topics.append(topic)
    return topics


In [13]:
pickup_topic_words(model_topic, topn=30)[0][:5]


[('BERT', 0.03100991),
 ('し', 0.024096573),
 ('自然言語処理', 0.022761282),
 ('AI', 0.022514433),
 ('の', 0.015532182)]

In [14]:
def parse_topic(topics: list):
    words = []
    probs = []
    for w, p in topics:
        do_skip: bool = False
        do_skip |= bool(re.search(r"^[あ-ん]", w))    # です、ます などは、スキップ
        do_skip |= bool(re.search(r"[あ-ん]$", w))    # 動名詞 などは、スキップ
        do_skip |= w[0].isnumeric()                  # 数字から始まるラベルはスキップ
        if do_skip:
            continue
        words.append(w)
        probs.append(p)
    return words, probs


In [15]:
import numpy
import re


def estimate_topic_label(w2v, topics: list):
    proper_topics = {}
    topic_labels = []

    for idx_tpc, tpc in enumerate(topics):
        _words, _probs = parse_topic(tpc)
        words = [w for w in _words if w in w2v.wv]
        probs = [p for w, p in zip(_words, _probs) if w in w2v.wv]
        vectors = w2v.wv[words]
        probs = numpy.array(probs).reshape(-1, 1)
        topic_vector = (vectors * probs).sum(axis=0)    # 期待値ベクトル
        estimated_topic_labels = w2v.wv.similar_by_vector(topic_vector, topn=100)

        labels, similarities = parse_topic(estimated_topic_labels)
        topic_label = labels[0]
        similarity = similarities[0]

        if topic_label not in proper_topics:
            proper_topics[topic_label] = (idx_tpc, similarity, words, probs)
        topic_labels.append((topic_label, similarity))

    return topic_labels, proper_topics


In [16]:
# w2v : is already loaded
n_topic_words = 17      # to calculate the average over
n_topic = 10            # default topic numbers
topic_label_counter = {}
while True:
    pipe_topic = Pipeline(
        steps=[
            (TokenizerWord(use_stoppoes=True, use_orgform=True), None),
            (VectorizerBoW(), None),
            (TopicModel(n_topic=n_topic, n_epoch=2000), None),
        ],
        name="pipe_topic",
        do_print=False,
    )

    X: TextSequences = [contents]
    pipe_topic.fit(X)

    model_topic: TopicModel = pipe_topic.get_model(-1)
    topics = pickup_topic_words(model_topic, topn=n_topic_words)
    topic_labels, proper_topics = estimate_topic_label(w2v, topics)

    # ラベルをカウント
    for tpc in proper_topics:
        cnt = topic_label_counter.get(tpc, 0) + 1
        topic_label_counter[tpc] = cnt
    
    # ループの終了条件
    if len(proper_topics) >= n_topic:
        break
    if n_topic <= 2:
        print("possibly failed to specify the topic numbers")
        break

    # 状態/処理文脈を更新
    n_topic = len(proper_topics)

possibly failed to specify the topic numbers


In [17]:
for idx_tpc, (lbl, sim) in enumerate(topic_labels):
    print("-" * 100)
    print(f"topic[{idx_tpc}]: {lbl} : {topic_label_counter[lbl]} ({sim:0.3f})")
    print(" " * 4 + f" ... {[_t for _t, _s in topics[idx_tpc][:20]]}")


----------------------------------------------------------------------------------------------------
topic[0]: Jabberwacky : 3 (0.781)
     ... ['し', 'BERT', 'AI', '社', 'いっ', 'たち', 'ない', 'コーパス', '技術', 'ある程度', '新しい', '統一', 'くれる', '視聴者', '人々', '画像', '感じ']
----------------------------------------------------------------------------------------------------
topic[1]: Jabberwacky : 3 (0.784)
     ... ['し', 'AI', 'BERT', '社', 'いっ', 'ある程度', 'ない', '新しい', 'コーパス', '人々', '視聴者', '技術', 'くれる', 'たち', '以前', '感じ', 'はじめて']


In [18]:
print("estimated n_topic:", len(proper_topics))

# print("label: (topic index, similarity to the expectation vector, words, probabilities)")

# 実際に期待値ベクトルを算出するときに使った単語を可視化
for lbl, v in proper_topics.items():
    tpc_idx, sim, words, probs = v
    print(f"[{tpc_idx:02d}]: {lbl}: {words}")


estimated n_topic: 1
[00]: Jabberwacky: ['AI', '社', 'コーパス', '技術', '統一', '視聴者', '人々', '画像']


In [19]:
print("\n".join(contents))

日本語でBERTが動くイメージを知ってほしい。
『BERTによる自然言語処理入門』の著者に聞く | Ledge.ai AI（人工知能）関連メディア メールマガジン登録 情報提供はこちら About Contact Hot Topics 注目のキーワード Fore- 見えない変化を可視化する。
見えている世界が動き出す。
Dataiku DATAFLUCT キカガク AI開発の現場 DataRobot JDLA DL for DX 沖電気工業 Gravio 簡単IoTとAI実装 ビジネス 学習 開発 エンタメ ラジオ Podcast 特集：新型コロナとデータ分析 特集：それ私が作りました！ Article Theme テーマ別カテゴリ プレスリリース イベント レッジのAI開発・コンサルティング AI・データ活用研修 AI（人工知能）ニュース インタビュー イベントレポート AI活用事例 やってみた 文系大学生がAI講座受けてみた ナレッジ AI本書評 ツール紹介 サービス紹介 マーケティング マーケティングオートメーション（MA） PR TIMES イベント告知 AI TALK NIGHT Ledge.ai Webinar 海外AIニュース速報 AI（人工知能）ニュースまとめ Technology 技術別カテゴリ ノーコード データサイエンス DX プログラミング エッジAI 自動運転 AI（人工知能） 注目テクノロジー 自然言語処理 画像認識・生成 チャットボット 音声認識・合成 予測 About Contact TOP > Article Theme > インタビュー > 日本語でBERTが動くイメージを知ってほしい。
『BERTによる自然言語処理入門』の著者に聞く 2021 08 25 Wed インタビュー 日本語でBERTが動くイメージを知ってほしい。
『BERTによる自然言語処理入門』の著者に聞く Tweet 6月に販売開始した、ストックマーク株式会社の機械学習エンジニアによる共著『BERTによる自然言語処理入門：Transformersを使った実践プログラミング』（オーム社）。
売れ行きも好調で、発売日の前から重版が決定していた。
本書はデータセットの処理から、BERTのファインチューニング（BERTを特定の言語タスクに特化させるための学習）、性能の評価

## LDA の可視化

In [20]:
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()


/usr/local/lib/python3.10/dist-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload


In [21]:
lda = model_topic.model
bow = pipe_topic[:-1](X)
vis = pyLDAvis.gensim_models.prepare(lda, corpus=bow, dictionary=model_bow.vocab)
pyLDAvis.display(vis)


IndexError: index 703 is out of bounds for axis 1 with size 703